In [1]:
import pandas as pd
import re
from collections import defaultdict
import spacy
from glob import glob
import unidecode
import os
import string
import numpy as np
import pickle as pkl
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import pickle as pkl

import warnings
warnings.filterwarnings("ignore")

# Text pre-processing

In [2]:
PATH = '../'

PROCESSED_DATA_PATH = os.path.join(PATH, 'data/processed/')

DF_FAKE_PATH = os.path.join(PROCESSED_DATA_PATH, 'df_fake_clean.pkl')
DF_LEGIT_PATH = os.path.join(PROCESSED_DATA_PATH, 'df_legit_clean.pkl')

df_fake = pkl.load(open(DF_FAKE_PATH, 'rb'))

df_legit = pkl.load(open(DF_LEGIT_PATH, 'rb'))

df = pd.concat((df_fake, df_legit), axis=0)

df.rename({'TEXT_CLEAN': 'X', 'FAKE': 'y'}, axis=1, inplace=True)

df.head()

,DATE,y,X,TITLE_CLEAN,TEXT_LEN_CHAR,TEXT_LEN_TOKEN,TITLE_LEN_CHAR,TITLE_LEN_TOKEN
0,03/08/2019,1,vamos assinar peticao cassacao mandato bolsona...,peticao impeachment bolsonaro precisa milhoes ...,328,36,81,9
1,04/08/2019,1,lula vitima golpe politico merece estar preso ...,peticao lula livre contribui liberdade preside...,242,30,54,7
2,05/08/2019,1,professor contou dilma matou mario kozel filho...,mario kozel filho assassinado dilma tiros,170,24,41,6
3,03/08/2019,1,vergonha presidente oab mentiu pai morto milit...,felipe santa cruz presidente oab mentiu sobre ...,158,25,82,13
4,06/08/2019,1,partido diabolico bandidos abrindo buracos est...,esquerda abrindo buracos estradas nordeste con...,155,19,59,7


In [3]:
df.shape

(16950, 8)

# Training word2vec model

In [4]:
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

## Join terms that are commonly found together

e.g.: belo_horizonte

In [5]:
sent = [text.split() for text in df['X']]
phrases = Phrases(sent, min_count=30, progress_per=10000)

In [6]:
df['X_phrased'] = [phrases[s] for s in sent]

In [7]:
w2v_model = Word2Vec(df['X_phrased'],
                    min_count=5,
                    window=9,
                    size=100,
                    workers=3,
                    iter=50)

In [49]:
wrd = 'bolsonaro'
print("Palavra: {}".format(wrd))
print("Palavras semelhantes:")
if wrd in w2v_model:
    for item in w2v_model.wv.most_similar(wrd):
        print('- {}'.format(item[0]))

Palavra: bolsonaro
Palavras semelhantes:
- jair_bolsonaro
- bolsonaro_psl
- mourao
- presidente_eleito
- capitao_reformado
- capitao_reserva
- eduardo_bolsonaro
- candidato_psl
- bolsonarista
- bolsonaro_psc


In [ ]:
from tqdm import tqdm_notebook

In [ ]:
all_df = pd.DataFrame()

for text in tqdm_notebook(df['X_phrased']):
    aux = pd.DataFrame()
    for word in text:
        try:
            word_vec = w2v_model[word]
            aux = aux.append(pd.Series(word_vec), ignore_index=True)
        except:
            pass
        doc_vec = aux.mean()
    all_df = all_df.append(doc_vec, ignore_index=True)

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler

In [19]:
pkl.dump(all_df, open('../data/processed/text_w2v.pkl', 'wb'))

# all_df = pkl.load(open('../data/processed/text_w2v.pkl', 'rb'))

In [20]:
rus = RandomUnderSampler(random_state=42)
resampled_X, resampled_y = rus.fit_resample(all_df, df['y'])

In [21]:
train_x, test_x, train_y, test_y = train_test_split(resampled_X,
                                                   resampled_y,
                                                   test_size = 0.2,
                                                   random_state = 1)

In [22]:
train_x.shape, test_x.shape, train_y.shape, test_y.shape

((2001, 100), (501, 100), (2001,), (501,))

In [45]:
model = RandomForestClassifier(n_estimators=1000)
model.fit(train_x, train_y)
test_pred = model.predict(all_df)

In [46]:
print(classification_report(df['y'], test_pred))

              precision    recall  f1-score   support

           0       1.00      0.95      0.97     15699
           1       0.59      0.98      0.74      1251

   micro avg       0.95      0.95      0.95     16950
   macro avg       0.79      0.96      0.85     16950
weighted avg       0.97      0.95      0.95     16950



In [47]:
print(confusion_matrix(df['y'], test_pred))

[[14836   863]
 [   21  1230]]
